##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# In-context Learning with Gemma 3

Large language models can learn new tasks or follow instructions through **in-context learning**, where examples within the input prompt guide them. This differs from traditional fine-tuning, which updates the model's parameters. Instead, it uses the model's existing knowledge and pattern recognition.

The input prompt typically shows examples of the desired input-output format or task. This context allows the model to understand and respond appropriately to new, unseen inputs within the same prompt. The quality and relevance of these examples, along with the model's overall capacity, influence its effectiveness.

Gemma 3 offers a much larger context window compared to its predecessor, with the **1B** parameter model supporting **32k tokens** and the **4B, 12B and 27B** models handling up to **128k tokens**, up from the previous 8k token limit.

The larger context windows in models like Gemma 3 are particularly beneficial for in-context learning, as they allow for more examples and more complex instructions to be included within a single prompt, potentially leading to improved performance on a wider range of tasks without explicit fine-tuning.

In this notebook, we'll apply in-context learning to replicate the result of our previous "[Translator of Old Korean Literature](https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_2]Translator_of_Old_Korean_Literature.ipynb)" fine-tuning example.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_3]In-context_Learning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>


## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.


### Gemma setup on Kaggle
To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run the Gemma 3 1B or 4B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [ ]:
import os
from google.colab import userdata # `userdata` is a Colab API.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### Install dependencies

This tutorial uses the [Gemma library](https://gemma-llm.readthedocs.io/) for JAX. Gemma library is a Python package built as an extension of [JAX](https://github.com/jax-ml/jax), letting you use the performance advantages of the JAX framework with dramatically less code.

In [ ]:
!pip install -U datasets
!pip install -q git+https://github.com/google-deepmind/gemma.git

## Load and prepare the Gemma model

1. Load the Gemma model with [`kagglehub.model_download`](https://github.com/Kaggle/kagglehub/blob/bddefc718182282882b72f814d407d89e5d178c4/src/kagglehub/models.py#L12), which takes three arguments:

- `handle`: The model handle from Kaggle
- `path`: (Optional string) The local path
- `force_download`: (Optional boolean) Forces to re-download the model

In [ ]:
# @markdown ---
# @markdown Choose Gemma model
GEMMA_VARIANT = 'gemma3-1b-it' # @param ['gemma3-1b-it', 'gemma3-4b-it'] {type:"string"}

import kagglehub

GEMMA_PATH = kagglehub.model_download(f'google/gemma-3/flax/{GEMMA_VARIANT}')
# example: download tokenizer only, force re-download
#GEMMA_PATH = kagglehub.model_download(f'google/gemma-3/flax/{GEMMA_VARIANT}', path="tokenizer.model", force_download=True)
print('GEMMA_PATH:', GEMMA_PATH)

import os
CKPT_PATH = os.path.join(GEMMA_PATH, GEMMA_VARIANT)
TOKENIZER_PATH = os.path.join(GEMMA_PATH, 'tokenizer.model')
print('CKPT_PATH:', CKPT_PATH)
print('TOKENIZER_PATH:', TOKENIZER_PATH)

from gemma import gm

if "gemma3-1b" in GEMMA_VARIANT:
    model = gm.nn.Gemma3_1B()
elif "gemma3-4b" in GEMMA_VARIANT:
    model = gm.nn.Gemma3_4B()
else:
    raise ValueError(f"Unknown GEMMA_VARIANT: {GEMMA_VARIANT}")

params = gm.ckpts.load_params(CKPT_PATH)
tokenizer = gm.text.Gemma3Tokenizer(TOKENIZER_PATH)

# @markdown Adjust cache size for in-context learning
CACHE_LENGTH = 12*1024 # @param
# @markdown Number of of tokens to generate output
MAX_NEW_TOKENS = 256 # @param
# @markdown ---

sampler = gm.text.Sampler(
    model=model,
    params=params,
    tokenizer=tokenizer,
    cache_length=CACHE_LENGTH,
)


GEMMA_PATH: /kaggle/input/gemma-3/flax/gemma3-1b-it/1
CKPT_PATH: /kaggle/input/gemma-3/flax/gemma3-1b-it/1/gemma3-1b-it
TOKENIZER_PATH: /kaggle/input/gemma-3/flax/gemma3-1b-it/1/tokenizer.model


## Load Dataset

Here's [the dataset](https://huggingface.co/datasets/bebechien/HongGildongJeon) from Hong Gildong jeon (Korean: 홍길동전), which is a Korean novel written during the Joseon Dynasty. The [original source](https://ko.wikisource.org/wiki/%ED%99%8D%EA%B8%B8%EB%8F%99%EC%A0%84_36%EC%9E%A5_%EC%99%84%ED%8C%90%EB%B3%B8) is in public domain. You will use a [modern translation](https://ko.wikisource.org/wiki/%ED%99%8D%EA%B8%B8%EB%8F%99%EC%A0%84_36%EC%9E%A5_%EC%99%84%ED%8C%90%EB%B3%B8/%ED%98%84%EB%8C%80%EC%96%B4_%ED%95%B4%EC%84%9D) in a [creative commons license](https://creativecommons.org/licenses/by-sa/4.0/), translated by `직지프로`.

To simplify the task, you will adopt the following structure for fine-tuning the model. The model will generate contemporary Korean text based on the user's input in [Early Hangul](https://en.wikipedia.org/wiki/Origin_of_Hangul).

```
<start_of_turn>user\n
됴션국셰둉ᄃᆡ왕즉위십오연의홍희문밧긔ᄒᆞᆫᄌᆡ상이잇스되<end_of_turn>\n
<start_of_turn>model\n
조선국 세종대왕 즉위 십오년에 홍회문 밖에 한 재상이 있으되,<end_of_turn>
```

> NOTE: korean text means, In the fifteenth year of the reign of King Sejong of Joseon, there was a prime minister outside Honghoemun Gate.

You'll start by loading the complete 'train' dataset and converting its features to NumPy arrays. Then you'll use few-shot prompting to include the dataset as an in-context learning example, utilizing the previously defined variables `in_context_limit = CACHE_LENGTH - MAX_NEW_TOKENS`

In [ ]:
from datasets import load_dataset

ds = load_dataset(
    "bebechien/HongGildongJeon",
    split="train",
)
print(ds)
data = ds.with_format(
    "np", columns=["original", "modern translation"], output_all_columns=False
)

in_context_learning = ""
in_context_length = 0

in_context_limit = CACHE_LENGTH - MAX_NEW_TOKENS

for x in data:
    item = f"<start_of_turn>user\n{x['original']}<end_of_turn>\n<start_of_turn>model\n{x['modern translation']}<end_of_turn>\n"
    length = len(tokenizer.encode(item))
    if in_context_length + length < in_context_limit:
        in_context_length += length
        in_context_learning += item
    else:
        break

print(in_context_length)
print(in_context_learning)

Dataset({
    features: ['original', 'modern translation'],
    num_rows: 447
})
11733
<start_of_turn>user
됴션국셰둉ᄃᆡ왕즉위십오연의홍희문밧긔ᄒᆞᆫᄌᆡ상이잇스되셩은홍이요명은문이니위인이쳥염강직ᄒᆞ여덩망이거록ᄒᆞ니당셰의영웅이라일직용문의올나벼살이할림의쳐ᄒᆞ엿더니명망이됴졍의읏듬되ᄆᆡ젼하그덕망을승이녀긔ᄉᆞ벼살을도도와이조판셔로좌으졍을ᄒᆞ이시니승상이국은을감동ᄒᆞ야갈츙보국ᄒᆞ니ᄉᆞ방의일이업고도젹이업스ᄆᆡ시화연풍ᄒᆞ여나라이ᄐᆡ평ᄒᆞ더라<end_of_turn>
<start_of_turn>model
조선국 세종대왕 즉위 십오년에 홍회문 밖에 한 재상이 있으되, 성은 홍이요, 명은 문이니, 위인이 청렴강직하여 덕망이 거룩하니 당세의 영웅이라. 일찍 용문에 올라 벼슬이 한림에 처하였더니 명망이 조정의 으뜸 되매, 전하 그 덕망을 승히 여기사 벼슬을 돋우어 이조판서로 좌의정을 하게 하시니, 승상이 국은을 감동하여 갈충보국하니 사방에 일이 업고 도적이 없으매 시화연풍하여 나라가 태평하더라.<end_of_turn>
<start_of_turn>user
일일은승상난간의비겨잠ᄀᆞᆫ조의더니ᄒᆞᆫ풍이긜을인도ᄒᆞ여ᄒᆞᆫ고듸다다르니쳥산은암암ᄒᆞ고녹슈난양양ᄒᆞᆫ듸셰류쳔만ᄀᆞ지녹음이파ᄉᆞᄒᆞ고황금갓ᄐᆞᆫᄭᅬᄭᅩ리난춘흥을희롱ᄒᆞ여냥뉴간의왕ᄂᆡᄒᆞ며긔화요초만발ᄒᆞᆫᄃᆡ쳥학ᄇᆡᆨ학이며비취공작이춘광을ᄌᆞ랑ᄒᆞ거날승상이경물을귀경ᄒᆞ며졈졈드러가니만쟝졀벽은하날의다엇고구뷔구뷔벽계슈난골골이폭포되어오운이어러엿난ᄃᆡ길이ᄭᅳᆫ쳐갈바을모로더니문득쳥용이물결을혜치고머리을드러고함ᄒᆞ니산학이믄허지난듯ᄒᆞ더니그용이입을버리고긔운을토ᄒᆞ여승상의입으로드러뵈거날ᄭᆡ다르니평ᄉᆡᆼᄃᆡ몽이라ᄂᆡ염의혜아리되피련군ᄌᆞ을나희리라ᄒᆞ여즉시ᄂᆡ당의드러ᄀᆞ시비을믈이치고부인을익그러취침코져ᄒᆞ니부인이졍ᄉᆡᆨ왈승상은국지ᄌᆡ상이라쳬위존즁ᄒᆞ시거날ᄇᆡᆨ쥬의졍실의드러와노류장화갓치ᄒᆞ시니ᄌᆡ상의쳬면이어ᄃᆡ잇난잇ᄀᆞ승상이ᄉᆡᆼ각ᄒᆞ신직말ᄉᆞᆷ은당연ᄒᆞ오나ᄃᆡ몽을허송할가ᄒᆞ야몽ᄉᆞ을이르지아니

## Generate Output

Define helper function and test cases

In [ ]:
import time
tick_start = 0

def tick():
    global tick_start
    tick_start = time.time()

def tock():
    print(f"TOTAL TIME ELAPSED: {time.time() - tick_start:.2f}s")

test_cases = [
    "ᄃᆡ작ᄒᆞ여그ᄭᅩᆺ치흣터지거ᄂᆞᆯ",
    "금두겁이품의드러뵈니일졍ᄌᆡᄌᆞᄅᆞᆯ나흐리로다ᄒᆞ더니과연그ᄃᆞᆯ부터잉ᄐᆡᄒᆞ여십삭이차니",
    "이ᄯᆡᄂᆞᆫᄉᆞ월초팔일이라이날밤의오ᄉᆡᆨ구룸이집을두루고향ᄂᆡ진동ᄒᆞ며션녀ᄒᆞᆫᄡᅣᆼ이촉을들고드러와김ᄉᆡᆼᄃᆞ려니르ᄃᆡ",
    "ᄌᆡ히길너텬졍을어긔지말으소셔이아희ᄇᆡ필은낙양니샹셔집아ᄌᆡ니",
]

### Without In-context Learning

The output below shows that the model doesn't recognize word or phrase and has no capability to translate Early Hangul.

In [ ]:
chat_prompt = "<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"

for test_case in test_cases:
    tick()
    output = sampler.sample(chat_prompt.format(prompt=test_case), max_new_tokens=MAX_NEW_TOKENS)
    print(output)
    tock()
    print('-'*80)


క్షమించండి, నేను ఆ ప్రశ్నకు సమాధానం ఇవ్వలేను. నేను హానికరమైన లేదా ప్రమాదకరమైన ప్రతిస్పందనలను ఉత్పత్తి చేయడానికి రూపొందించబడలేదు.<end_of_turn>
TOTAL TIME ELAPSED: 29.95s
--------------------------------------------------------------------------------
이 문장은 아름다운 한국어 시조입니다. 

**번역:**

금두겁이 품에 안겨 일정한 자리에서,
드러뵈니, 놀라움이 일었습니다.
과연 그 드러내니, 낯선 곳에서 시작되어,
다시 다녀보니, 사삭이 차니.

**해석:**

금두겁이 덮여 있는 자리에서,
어떤 놀라운 일이 일어났는지,
그 드러내니, 낯선 곳에서 시작되어,
다시 돌아와보니, 사삭이 차니.

**추가 설명:**

*   **금두겁이:** 금을 덮여 있는, 덮여진 상태를 의미합니다.
*   **일졍ᄌᆡᄌᆞ:** (일정) 낯선, 낯선 장소
*   **ᄅᆞᆯ나흐리로다ᄒᆞ더니:** (드러뵈니) 드러나다, 튀어나다
*   **과연그ᄃᆞᆯ부터잉:** (그 드러내니) 그 모습, 그 모습이
*   **ᄐᆡᄒᆞ더니:** (다녀보니)
TOTAL TIME ELAPSED: 23.34s
--------------------------------------------------------------------------------
이 문장은 매우 어색하고, 의미를 파악하기 어렵습니다. 

다만, 몇 가지 추측을 해볼 수 있습니다.

* **문맥:** 이 문장은 아마도 특정 상황이나 이야기를 암시하는 것 같습니다.
* **어색한 표현:** "이ᄯᆡᄂᆞᆫᄉᆞ월초팔일이라이날밤의오ᄉᆡᆨ구룸이집을두루고향ᄂᆡ진동ᄒᆞ며션녀ᄒᆞᆫᄡᅣᆼ이촉을들고드러와김ᄉᆡᆼᄃᆞ려니르" 와 같은 표현은 문법적으로 어색하고 의미가 불분명합니다.
* **가능성:** 이 문장은 아마도 특정 장소나 상황에 대한 묘사일 수 있습니

### With In-context Learning

In [ ]:
in_context_learning_prompt = in_context_learning + "<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"

for test_case in test_cases:
    tick()
    output = sampler.sample(in_context_learning_prompt.format(prompt=test_case), max_new_tokens=MAX_NEW_TOKENS)
    print(output)
    tock()
    print('-'*80)


대감께서 굳이 작고 굳이 굳치흣터지거니와.<end_of_turn>
TOTAL TIME ELAPSED: 42.74s
--------------------------------------------------------------------------------
금두겁이품의 드러뵈니 일정자 굳이 품에 붙어 있네.<end_of_turn>
TOTAL TIME ELAPSED: 43.40s
--------------------------------------------------------------------------------
이 밤 초팔일이 라이날 밤의 오수구룸이 집을 두루고 향냄이 동동 춤추며, 여성의 얼굴을 들고 드러와 김수영이 려니르.<end_of_turn>
TOTAL TIME ELAPSED: 46.50s
--------------------------------------------------------------------------------
길동이 길 너방에 말하지 말소셔이아희 필은낙양니샹셔집아지말소<end_of_turn>
TOTAL TIME ELAPSED: 44.33s
--------------------------------------------------------------------------------


For your reference, please see the following text, which has been translated by a human.

---

Early Hangul :
`금두겁이품의드러뵈니일졍ᄌᆡᄌᆞᄅᆞᆯ나흐리로다ᄒᆞ더니과연그ᄃᆞᆯ부터잉ᄐᆡᄒᆞ여십삭이차니`

Result via in-context Learning : `금두겁이품의 드러뵈니 일정자 굳이 품에 붙어 있네.<end_of_turn>`

Human translation :
```
금두꺼비가 품에 드는 게 보였으니 얼마 안 있어 자식을 낳을 것입니다.

하였다. 과연 그 달부터 잉태하여 십삭이 차니
```

> Note: Korean text means, “I saw a golden toad in her arms, so it won’t be long before she gives birth to a child.” Indeed, she conceived from that month and was ten months old.

---

Early Hangul : `이ᄯᆡᄂᆞᆫᄉᆞ월초팔일이라이날밤의오ᄉᆡᆨ구룸이집을두루고향ᄂᆡ진동ᄒᆞ며션녀ᄒᆞᆫᄡᅣᆼ이촉을들고드러와김ᄉᆡᆼᄃᆞ려니르ᄃᆡ`

Result via in-context Learning : `이 밤 초팔일이 라이날 밤의 오수구룸이 집을 두루고 향냄이 동동 춤추며, 여성의 얼굴을 들고 드러와 김수영이 려니르.<end_of_turn>`

Human translation :
```
이 때는 사월 초파일이었다. 이날 밤에 오색구름이 집을 두르고 향내 진동하며 선녀 한 쌍이 촉을 들고 들어와 김생더러 말하기를,
```

> Note: Korean text means, At this time, it was the 8th of April. On this night, with five-colored clouds surrounding the house and the scent of incense vibrating, a pair of fairies came in holding candles and said to Kim Saeng,

Although the translation is not flawless, it provides a decent initial draft. The results are remarkable, considering that the datasets are limited to a single book. Enhancing the diversity of data sources will likely improve the translation quality.

## Conclusion

Why this works? Because the model is leveraging their extensive pre-training knowledge and the power of the transformer architecture to recognize patterns, infer concepts, and adapt their behavior to new tasks presented directly within the input context.

When deciding between "Fine-tuning" and "In-context learning" with LLM, consider following factors:

### Use Fine-tuning When:

* **High Performance and Accuracy are Crucial:** For tasks requiring the best possible performance and accuracy in a specific domain, fine-tuning is generally preferred. It allows the model to deeply adapt its parameters to the nuances of your data.
* **Domain-Specific Tasks:** If your application focuses on a narrow domain with specific vocabulary, style, or knowledge (e.g., legal documents, medical records, financial reports), fine-tuning can significantly improve the model's understanding and output quality.
* **Consistent Output Format is Needed:** Fine-tuning can enforce a specific output format or structure more reliably than in-context learning.
* **Cost-Effective Inference in the Long Run:** While fine-tuning requires upfront computational cost and labeled data, the resulting specialized model can often be smaller and more efficient for inference, leading to lower long-term costs, especially for high-volume applications.

### Use In-Context Learning When:

* **Rapid Prototyping and Experimentation:** In-context learning is excellent for quickly testing the capabilities of an LLM on a new task without the time and resource investment of fine-tuning.
* **Flexibility and Task Switching:** If your application needs to handle a wide variety of tasks or adapt to changing requirements on the fly, in-context learning allows you to guide the model with different prompts and examples without retraining.
* **Limited or No Labeled Data:** When you don't have a large, labeled dataset for your specific task, in-context learning can be a viable alternative by providing a few relevant examples directly in the prompt.
* **Utilizing General Capabilities of Large Models:** If the task can be reasonably accomplished by leveraging the broad knowledge and general language understanding of a powerful pre-trained model with well-crafted prompts.

### Hybrid Approach:

It's also possible to combine both techniques. You could fine-tune a model on a broader domain and then use in-context learning with specific examples to further refine its behavior for particular sub-tasks within that domain.

### In summary:

* **Fine-tuning** is for **specialization, high performance, and long-term efficiency** when you have sufficient labeled data and a well-defined task.
* **In-context learning** is for **flexibility, rapid experimentation, and handling diverse or low-data scenarios** by leveraging the general capabilities of large models through effective prompting.
